# Dummy Operation

In [145]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import numpy as np
from scipy import stats
from tqdm import tqdm
import pingouin as pg
from colorama import Fore, Style
from scipy.stats import spearmanr



tqdm.pandas()

In [146]:
# pd.options.mode.dtype_backend = 'pyarrow'
plt.rcParams["figure.figsize"] = (10, 6)
pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
pd.options.display.float_format = '{:.2f}'.format
sns.set_style("whitegrid")

In [147]:
df = pd.read_csv('outliers_handled.csv', engine='pyarrow')

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24807 entries, 0 to 24806
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   make_model               24807 non-null  object 
 1   make                     24807 non-null  object 
 2   model                    24807 non-null  object 
 3   price                    24807 non-null  float64
 4   body_type                24807 non-null  object 
 5   age                      24807 non-null  float64
 6   fuel                     24807 non-null  object 
 7   country                  24807 non-null  object 
 8   type                     24807 non-null  object 
 9   warranty                 24807 non-null  object 
 10  mileage                  24807 non-null  float64
 11  gearbox                  24807 non-null  object 
 12  seller                   24807 non-null  object 
 13  engine_size              24807 non-null  float64
 14  gears                 

In [149]:
df.columns

Index(['make_model', 'make', 'model', 'price', 'body_type', 'age', 'fuel',
       'country', 'type', 'warranty', 'mileage', 'gearbox', 'seller',
       'engine_size', 'gears', 'co2_emissions', 'drivetrain', 'cylinders',
       'extras', 'empty_weight', 'full_service_history', 'upholstery',
       'previous_owner', 'energy_efficiency_class', 'power_hp',
       'fuel_consumption_avg', 'address', 'latitude', 'longitude', 'cc', 'ss',
       'em'],
      dtype='object')

In [150]:
categoric_col = df.select_dtypes(include="object")
display(categoric_col.columns)

Index(['make_model', 'make', 'model', 'body_type', 'fuel', 'country', 'type',
       'warranty', 'gearbox', 'seller', 'drivetrain', 'extras',
       'full_service_history', 'upholstery', 'energy_efficiency_class',
       'address', 'cc', 'ss', 'em'],
      dtype='object')

In [151]:
print(df.shape)
df.drop(["address", "latitude", "longitude"], axis=1, inplace=True)
print(df.shape)

(24807, 32)
(24807, 29)


In [152]:
categoric_col = df.select_dtypes(include="object")
display(categoric_col.columns)

Index(['make_model', 'make', 'model', 'body_type', 'fuel', 'country', 'type',
       'warranty', 'gearbox', 'seller', 'drivetrain', 'extras',
       'full_service_history', 'upholstery', 'energy_efficiency_class', 'cc',
       'ss', 'em'],
      dtype='object')

In [153]:
print(Fore.BLUE + Style.BRIGHT + "Number of Uniques:\n" +
      Fore.GREEN + "****"*10 + Style.RESET_ALL + '\n' +
      str(categoric_col.nunique()))

Number of Uniques:
****************************************
make_model                  490
make                         13
model                       488
body_type                     6
fuel                          6
country                      13
type                          4
warranty                      2
gearbox                       3
seller                        2
drivetrain                    3
extras                     9027
full_service_history          2
upholstery                    2
energy_efficiency_class       2
cc                            3
ss                            3
em                            2
dtype: int64


In [154]:
# Encoding all categorical variable (except for "make", "body_type", "fuel_type", seller", drivetrain", "extras", "full_service_history", "upholstery", "energy_efficiency_class", "comfort_convenience_package", "entertainment_media_package", "safety_security_package") in our DataFrame.

df[["make_model_ordinal", "type_ordinal", "warranty_ordinal", "gearbox_ordinal",'country_ordinal']] = df[["make_model", "type", "warranty", "gearbox",'country']].apply(lambda x: pd.factorize(x)[0])
df.head()

,make_model,make,model,price,body_type,age,fuel,country,type,warranty,mileage,gearbox,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em,make_model_ordinal,type_ordinal,warranty_ordinal,gearbox_ordinal,country_ordinal
0,Mercedes-Benz A 160,Mercedes-Benz,A 160,16950.00,Compact,6.00,Diesel,Spain,Used,Yes,120200.00,Manual,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard,0,0,0,0,0
1,Mercedes-Benz EQE 350,Mercedes-Benz,EQE 350,80900.00,Compact,0.00,Electric,Spain,Pre-registered,Yes,5000.00,Automatic,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium,1,1,0,1,0
2,Mercedes-Benz A 45 AMG,Mercedes-Benz,A 45 AMG,69900.00,Compact,2.00,Benzine,Spain,Used,Yes,18900.00,Automatic,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard,2,0,0,1,0
3,Mercedes-Benz A 35 AMG,Mercedes-Benz,A 35 AMG,46990.00,Compact,2.00,Benzine,Spain,Used,Yes,18995.00,Automatic,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard,3,0,0,1,0
4,Mercedes-Benz A 45 AMG,Mercedes-Benz,A 45 AMG,16800.00,Compact,7.00,Diesel,Spain,Used,Yes,197000.00,Automatic,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard,2,0,0,1,0


In [155]:
make_model_ordinal = df.pop("make_model_ordinal")
type_ordinal = df.pop("type_ordinal")
warranty_ordinal = df.pop("warranty_ordinal")
gearbox_ordinal = df.pop("gearbox_ordinal")
country_ordinal = df.pop("country_ordinal")

# Let's insert the columns we have popp
df.insert(1, 'make_model_ordinal', make_model_ordinal)
df.insert(6, "type_ordinal", type_ordinal)
df.insert( 8, "warranty_ordinal", warranty_ordinal)
df.insert(11, "gearbox_ordinal", gearbox_ordinal)
df.insert(13, "country_ordinal", country_ordinal)


df.head()

,make_model,make_model_ordinal,make,model,price,body_type,type_ordinal,age,warranty_ordinal,fuel,country,gearbox_ordinal,type,country_ordinal,warranty,mileage,gearbox,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em
0,Mercedes-Benz A 160,0,Mercedes-Benz,A 160,16950.00,Compact,0,6.00,0,Diesel,Spain,0,Used,0,Yes,120200.00,Manual,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard
1,Mercedes-Benz EQE 350,1,Mercedes-Benz,EQE 350,80900.00,Compact,1,0.00,0,Electric,Spain,1,Pre-registered,0,Yes,5000.00,Automatic,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium
2,Mercedes-Benz A 45 AMG,2,Mercedes-Benz,A 45 AMG,69900.00,Compact,0,2.00,0,Benzine,Spain,1,Used,0,Yes,18900.00,Automatic,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard
3,Mercedes-Benz A 35 AMG,3,Mercedes-Benz,A 35 AMG,46990.00,Compact,0,2.00,0,Benzine,Spain,1,Used,0,Yes,18995.00,Automatic,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard
4,Mercedes-Benz A 45 AMG,2,Mercedes-Benz,A 45 AMG,16800.00,Compact,0,7.00,0,Diesel,Spain,1,Used,0,Yes,197000.00,Automatic,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard


In [156]:
print(df.shape)
df.drop(["make_model", "type", "warranty", 'gearbox','country'], axis=1, inplace=True)
print(df.shape)

df.head()

(24807, 34)
(24807, 29)


,make_model_ordinal,make,model,price,body_type,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em
0,0,Mercedes-Benz,A 160,16950.00,Compact,0,6.00,0,Diesel,0,0,120200.00,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard
1,1,Mercedes-Benz,EQE 350,80900.00,Compact,1,0.00,0,Electric,1,0,5000.00,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium
2,2,Mercedes-Benz,A 45 AMG,69900.00,Compact,0,2.00,0,Benzine,1,0,18900.00,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard
3,3,Mercedes-Benz,A 35 AMG,46990.00,Compact,0,2.00,0,Benzine,1,0,18995.00,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard
4,2,Mercedes-Benz,A 45 AMG,16800.00,Compact,0,7.00,0,Diesel,1,0,197000.00,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard


In [157]:
df.dtypes

make_model_ordinal           int64
make                        object
model                       object
price                      float64
body_type                   object
type_ordinal                 int64
age                        float64
warranty_ordinal             int64
fuel                        object
gearbox_ordinal              int64
country_ordinal              int64
mileage                    float64
seller                      object
engine_size                float64
gears                      float64
co2_emissions              float64
drivetrain                  object
cylinders                  float64
extras                      object
empty_weight               float64
full_service_history        object
upholstery                  object
previous_owner             float64
energy_efficiency_class     object
power_hp                   float64
fuel_consumption_avg       float64
cc                          object
ss                          object
em                  

In [158]:
df.extras.str.count(",").add(1)
#df["extras"].apply(lambda x: x.count(",")).add(1)

0        1
1        6
2        1
3        1
4        1
        ..
24802    1
24803    2
24804    3
24805    2
24806    4
Name: extras, Length: 24807, dtype: int64

In [159]:
df["num_of_extras"] = df["extras"].apply(lambda x: x.count(",")).add(1)

df.head()

,make_model_ordinal,make,model,price,body_type,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,extras,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em,num_of_extras
0,0,Mercedes-Benz,A 160,16950.00,Compact,0,6.00,0,Diesel,0,0,120200.00,Dealer,1461.00,5.00,126.00,Front,4.00,Alloy wheels,1295.00,No,Cloth,2.00,efficient,90.00,5.45,premium,Safety Security Premium,Entertainment Media Standard,1
1,1,Mercedes-Benz,EQE 350,80900.00,Compact,1,0.00,0,Electric,1,0,5000.00,Dealer,1.00,1.00,0.00,4WD,0.00,"Alloy wheels (21""), Shift paddles, Sport packa...",2355.00,No,Part/Full Leather,1.00,efficient,292.00,2.36,premium_plus,Safety Security Premium Plus,Entertainment Media Premium,6
2,2,Mercedes-Benz,A 45 AMG,69900.00,Compact,0,2.00,0,Benzine,1,0,18900.00,Dealer,1991.00,8.00,162.00,4WD,4.00,Alloy wheels,1555.00,No,Part/Full Leather,2.00,inefficient,421.00,8.40,premium,Safety Security Standard,Entertainment Media Standard,1
3,3,Mercedes-Benz,A 35 AMG,46990.00,Compact,0,2.00,0,Benzine,1,0,18995.00,Dealer,1991.00,7.00,169.00,4WD,4.00,Sport seats,1455.00,No,Part/Full Leather,2.00,inefficient,306.00,7.30,premium,Safety Security Standard,Entertainment Media Standard,1
4,2,Mercedes-Benz,A 45 AMG,16800.00,Compact,0,7.00,0,Diesel,1,0,197000.00,Dealer,2143.00,7.00,162.00,4WD,4.00,Alloy wheels,1545.00,No,Part/Full Leather,2.00,inefficient,136.00,4.90,premium,Safety Security Premium,Entertainment Media Standard,1


In [160]:
print(df.shape)
df.drop("extras", axis=1, inplace=True)
print(df.shape)

(24807, 30)
(24807, 29)


In [161]:
df["fuel"].value_counts()

fuel
Benzine     14913
Diesel       7742
Electric     1793
LPG/CNG       343
Hydrogen       13
Ethanol         3
Name: count, dtype: int64

In [162]:
df.loc[(df.fuel == 'Hydrogen') | (df.fuel == 'Ethanol')].index

Index([ 3112,  5063, 16754, 16767, 16799, 16817, 16819, 16998, 16999, 17019,
       17781, 21895, 21939, 22081, 22128, 22137],
      dtype='int64')

In [163]:
print(df.shape)
df.drop(index=df.loc[(df.fuel == 'Hydrogen') | (df.fuel == 'Ethanol')].index, inplace = True)
print(df.shape)

(24807, 29)
(24791, 29)


In [164]:
df["seller"].value_counts()

seller
Dealer            23323
Private seller     1468
Name: count, dtype: int64

In [165]:
df["drivetrain"].value_counts()

drivetrain
Front    19111
4WD       3581
Rear      2099
Name: count, dtype: int64

In [166]:
df["full_service_history"].value_counts()

full_service_history
No     13673
Yes    11118
Name: count, dtype: int64

In [167]:
df["upholstery"].value_counts()

upholstery
Cloth                13635
Part/Full Leather    11156
Name: count, dtype: int64

In [168]:
df["energy_efficiency_class"].value_counts()

energy_efficiency_class
efficient      15680
inefficient     9111
Name: count, dtype: int64

In [169]:
df["fuel"] = df["fuel"].map({"Benzine":0, "Diesel":1, "LPG/CNG":2, "Electric":3})
df["seller"] = df["seller"].map({"Private seller":0, "Dealer":1})
df["drivetrain"] = df["drivetrain"].map({"Rear":0, "Front":1, "4WD":2})
df["full_service_history"] = df["full_service_history"].map({"No":0, "Yes":1})
df["upholstery"] = df["upholstery"].map({"Cloth":0, "Part/Full Leather":1})
df["energy_efficiency_class"] = df["energy_efficiency_class"].map({"inefficient":0, "efficient":1})


In [170]:
df.sample(1)

,make_model_ordinal,make,model,price,body_type,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,ss,em,num_of_extras
22013,449,Hyundai,KONA,28980.00,Off-Road/Pick-up,2,0.00,1,0,1,1,1550.00,1,1598.00,7.00,133.00,2,4.00,1395.00,1,0,1.00,1,199.00,5.60,premium,Safety Security Premium,Entertainment Media Standard,3


In [171]:
df["cc"].value_counts()

cc
premium         15834
premium_plus     5945
standard         3012
Name: count, dtype: int64

In [172]:
df["cc"] = df["cc"].astype("category")
df["cc"] = df["cc"].cat.codes


In [173]:
df["cc"].value_counts()

cc
0    15834
1     5945
2     3012
Name: count, dtype: int64

In [174]:
from sklearn.preprocessing import LabelEncoder

In [175]:
# Creating an instance of LabelEncoder() and storing it in labelencoder variable/object
labelencoder = LabelEncoder()

# Applying LabelEncoder for the columns of "safety_security_package" & "entertainment_media_package"
df[["ss_labeled", "em_labeled"]] = df[["ss", "em"]].apply(labelencoder.fit_transform)
df[["ss", "ss_labeled", "em", "em_labeled"]]

,ss,ss_labeled,em,em_labeled
0,Safety Security Premium,0,Entertainment Media Standard,1
1,Safety Security Premium Plus,1,Entertainment Media Premium,0
2,Safety Security Standard,2,Entertainment Media Standard,1
3,Safety Security Standard,2,Entertainment Media Standard,1
4,Safety Security Premium,0,Entertainment Media Standard,1
...,...,...,...,...
24802,Safety Security Standard,2,Entertainment Media Standard,1
24803,Safety Security Premium,0,Entertainment Media Standard,1
24804,Safety Security Premium,0,Entertainment Media Standard,1
24805,Safety Security Premium Plus,1,Entertainment Media Premium,0


In [176]:
display(df[df["ss"] == "Safety Security Premium"][["ss", "ss_labeled"]].sample(5))
display(df[df["ss"] == "Safety Security Premium Plus"][["ss", "ss_labeled"]].sample(5))
display(df[df["ss"] == "Safety Security Standard"][["ss", "ss_labeled"]].sample(5))
display(df[df["em"] == "Entertainment Media Premium"][["em", "em_labeled"]].sample(5))
display(df[df["em"] == "Entertainment Media Standard"][["em", "em_labeled"]].sample(5))

,ss,ss_labeled
4214,Safety Security Premium,0
19621,Safety Security Premium,0
12773,Safety Security Premium,0
20257,Safety Security Premium,0
18269,Safety Security Premium,0


,ss,ss_labeled
37,Safety Security Premium Plus,1
7746,Safety Security Premium Plus,1
23865,Safety Security Premium Plus,1
19763,Safety Security Premium Plus,1
13771,Safety Security Premium Plus,1


,ss,ss_labeled
8453,Safety Security Standard,2
12626,Safety Security Standard,2
20841,Safety Security Standard,2
19495,Safety Security Standard,2
13425,Safety Security Standard,2


,em,em_labeled
23969,Entertainment Media Premium,0
24210,Entertainment Media Premium,0
15770,Entertainment Media Premium,0
20551,Entertainment Media Premium,0
1613,Entertainment Media Premium,0


,em,em_labeled
14128,Entertainment Media Standard,1
16943,Entertainment Media Standard,1
15837,Entertainment Media Standard,1
18455,Entertainment Media Standard,1
9882,Entertainment Media Standard,1


In [177]:
print(df.shape)
df.drop(["ss", "em"], axis=1, inplace=True)
print(df.shape)

(24791, 31)
(24791, 29)


In [178]:
print(df.shape)
df.drop(['model'], axis=1, inplace=True)
print(df.shape)

(24791, 29)
(24791, 28)


In [179]:
for column in ['make', 'body_type']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns] = dummies
    df.drop(column, axis=1, inplace=True)

In [180]:
df.sample(1)

,make_model_ordinal,price,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,Seat,Skoda,Toyota,Volvo,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
2375,185,12300.00,0,4.00,1,0,0,1,27450.00,1,1398.00,5.00,118.00,1,4.00,1163.00,1,0,2.00,0,90.00,5.10,1,9,0,0,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False


In [181]:
df = df.astype('float')

In [182]:
df.sample(1)

,make_model_ordinal,price,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,Seat,Skoda,Toyota,Volvo,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
1319,133.00,71999.00,0.00,9.00,1.00,1.00,1.00,1.00,123973.00,1.00,2987.00,7.00,295.00,2.00,6.00,2610.00,0.00,1.00,1.00,0.00,211.00,11.20,1.00,14.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [183]:
df.shape

(24791, 45)

## Export dataframe to csv file (dummy)

In [184]:
df.to_csv("dummied.csv", index=False)

In [ ]:
df_final = pd.read_csv("dummied.csv")
df_final.head().T

In [ ]:
df.columns

In [185]:
df1 = df.drop(['Dacia', 'Fiat', 'Ford', 'Hyundai', 'Mercedes-Benz', 'Nissan', 'Opel', 'Peugeot', 'Renault', 'Seat', 'Skoda', 'Toyota', 'Volvo'], axis=1)

df1.to_csv("dummied_make_model.csv", index=False)

pd.read_csv("dummied_make_model.csv")

,make_model_ordinal,price,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
0,0.00,16950.00,0.00,6.00,0.00,1.00,0.00,0.00,120200.00,1.00,1461.00,5.00,126.00,1.00,4.00,1295.00,0.00,0.00,2.00,1.00,90.00,5.45,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
1,1.00,80900.00,1.00,0.00,0.00,3.00,1.00,0.00,5000.00,1.00,1.00,1.00,0.00,2.00,0.00,2355.00,0.00,1.00,1.00,1.00,292.00,2.36,1.00,6.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
2,2.00,69900.00,0.00,2.00,0.00,0.00,1.00,0.00,18900.00,1.00,1991.00,8.00,162.00,2.00,4.00,1555.00,0.00,1.00,2.00,0.00,421.00,8.40,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
3,3.00,46990.00,0.00,2.00,0.00,0.00,1.00,0.00,18995.00,1.00,1991.00,7.00,169.00,2.00,4.00,1455.00,0.00,1.00,2.00,0.00,306.00,7.30,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
4,2.00,16800.00,0.00,7.00,0.00,1.00,1.00,0.00,197000.00,1.00,2143.00,7.00,162.00,2.00,4.00,1545.00,0.00,1.00,2.00,0.00,136.00,4.90,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24786,480.00,37600.00,0.00,3.00,0.00,3.00,1.00,0.00,40345.00,1.00,1969.00,8.00,0.00,2.00,0.00,2077.00,0.00,1.00,2.00,1.00,392.00,2.00,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24787,474.00,5499.00,0.00,18.00,1.00,0.00,1.00,4.00,145000.00,1.00,2435.00,5.00,217.00,1.00,5.00,1357.00,1.00,1.00,3.00,1.00,170.00,9.10,0.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24788,476.00,7300.00,0.00,11.00,0.00,1.00,0.00,2.00,96000.00,1.00,1560.00,6.00,99.00,1.00,4.00,1356.00,1.00,0.00,1.00,0.00,114.00,3.80,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24789,480.00,29900.00,0.00,5.00,0.00,0.00,1.00,4.00,94343.00,1.00,1969.00,8.00,149.00,1.00,4.00,1617.00,1.00,1.00,1.00,1.00,254.00,6.50,0.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00


In [186]:
df2 = df.drop(['make_model_ordinal'], axis=1)

df2.to_csv("dummied_make.csv", index=False)

pd.read_csv("dummied_make.csv")

,price,type_ordinal,age,warranty_ordinal,fuel,gearbox_ordinal,country_ordinal,mileage,seller,engine_size,gears,co2_emissions,drivetrain,cylinders,empty_weight,full_service_history,upholstery,previous_owner,energy_efficiency_class,power_hp,fuel_consumption_avg,cc,num_of_extras,ss_labeled,em_labeled,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,Seat,Skoda,Toyota,Volvo,Compact,Convertible,Coupe,Off-Road/Pick-up,Sedan,Station wagon
0,16950.00,0.00,6.00,0.00,1.00,0.00,0.00,120200.00,1.00,1461.00,5.00,126.00,1.00,4.00,1295.00,0.00,0.00,2.00,1.00,90.00,5.45,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
1,80900.00,1.00,0.00,0.00,3.00,1.00,0.00,5000.00,1.00,1.00,1.00,0.00,2.00,0.00,2355.00,0.00,1.00,1.00,1.00,292.00,2.36,1.00,6.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
2,69900.00,0.00,2.00,0.00,0.00,1.00,0.00,18900.00,1.00,1991.00,8.00,162.00,2.00,4.00,1555.00,0.00,1.00,2.00,0.00,421.00,8.40,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
3,46990.00,0.00,2.00,0.00,0.00,1.00,0.00,18995.00,1.00,1991.00,7.00,169.00,2.00,4.00,1455.00,0.00,1.00,2.00,0.00,306.00,7.30,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
4,16800.00,0.00,7.00,0.00,1.00,1.00,0.00,197000.00,1.00,2143.00,7.00,162.00,2.00,4.00,1545.00,0.00,1.00,2.00,0.00,136.00,4.90,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24786,37600.00,0.00,3.00,0.00,3.00,1.00,0.00,40345.00,1.00,1969.00,8.00,0.00,2.00,0.00,2077.00,0.00,1.00,2.00,1.00,392.00,2.00,0.00,1.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24787,5499.00,0.00,18.00,1.00,0.00,1.00,4.00,145000.00,1.00,2435.00,5.00,217.00,1.00,5.00,1357.00,1.00,1.00,3.00,1.00,170.00,9.10,0.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24788,7300.00,0.00,11.00,0.00,1.00,0.00,2.00,96000.00,1.00,1560.00,6.00,99.00,1.00,4.00,1356.00,1.00,0.00,1.00,0.00,114.00,3.80,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
24789,29900.00,0.00,5.00,0.00,0.00,1.00,4.00,94343.00,1.00,1969.00,8.00,149.00,1.00,4.00,1617.00,1.00,1.00,1.00,1.00,254.00,6.50,0.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
